In [2]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from collections import defaultdict
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity as cos_sim
%run TFIDF_Summary.ipynb


class Cluster_First_process(Tfidf_process) :
    def __init__(self, n, nb_words = 100) :
        self.nb_words = nb_words
        self.__class__ = type(self.__class__.__name__, (self.__class__,), {})
        self.__class__.__call__ = lambda x:x
        self.n = n
        self.__name__ = "Cluster_First("+str(self.n)+")"
    
    def summarize(self, corpus, docs_bias=None) :
        X = self.vectorizer.transform(corpus).toarray()
        #C = AgglomerativeClustering(n_clusters = self.n)
        C = KMeans(n_clusters = self.n, n_init = 3, max_iter = 20)
        
        Z = C.fit_predict(X)
        clu = defaultdict(list)
        for ind, cls in enumerate(Z) :
            clu[cls].append(ind)
        summ = []
        selected = set()
        total_length = 0
        while total_length < self.nb_words:
            for clus in clu.values() :
                if (len(clus)) == 0:
                    continue
                minInd = max(clus)
                for ind in clus :
                    if ind not in selected:
                        minInd = min(minInd, ind)
                summ.append(minInd)
                total_length += len(corpus[minInd].split())
                selected.add(minInd)
        return sorted(summ)